In [24]:
import numpy as np
import os
from keras.models import Model
import pandas as pd
import os
import matplotlib.pyplot
%matplotlib inline
import librosa
import lib.util as ut
import tensorflow as tf

## Load the model we want to make inferences with

In [54]:
model_path='./trained_models/model_paper/model_15epochs.h5'

## To divide audio files. When making inferences over whole audio files, we pass interval_step=sample_rate/2 to produce 50% overlapping chunks of 3 second audio

In [66]:
def divide_audio_file(path, intervals_seconds, interval_step, sample_rate=8000):
    """

    """
    # loads file and converts to the specified sample rate    
    audio, fs = librosa.load(path, sample_rate)
    
    audio_array = []
    for i in range(0, audio.shape[0], interval_step):   
        interval = audio[i:i+sample_rate*intervals_seconds]
#         print("interval from {} to {}".format(i, i+sample_rate*intervals_seconds))
        
        # if the last interval is shorter han the interval in seconds we define we are going to ignore it
        if interval.shape[0] < sample_rate*intervals_seconds:
            break
        else:
            if (not ut.is_silence(interval,thresold_samples=0.70)):
                audio_array.append(interval)
            else:
                print("Omitting chunk with silences in file {}".format(path))

    return np.array(audio_array)

## Like the original create_tf_records but creating numpy array's instead

In [67]:
def get_chunks_from_raw(audio_path, file_names_df, num_samples):
    
    x_data = []
    labels = []
    tf_record_files = 1  
    for file in os.listdir(audio_path):

        file_path = audio_path + "/" + file
        short_name = file.split(".")[0]

        # if the file is in the dataframe with the file names(train or test) we divide it, if not we ignore
        if short_name in file_names_df["0"].values:
            print("reading file {}".format(file))

            divided_file =  divide_audio_file(file_path, intervals_seconds, sample_rate)

            file_label = file_names_df[file_names_df["0"] == short_name]["class"].values[0]
            labels_array = np.ones(divided_file.shape[0]) * file_label
            
            x_data.extend(divided_file)
            labels.extend(labels_array)            
            
            if len(x_data) > num_samples:
                tf_record_files += 1
                break;
                        
        else:
            print("file {} not in the dataframe".format(file))
       
                 
    return (x_data,labels)



## makes inference of a whole audio by dividing it into overlapping (50%) chunks and predicting the majority of chunks class as the class of the file

In [109]:
def predict_one_audio(file_name,label):
    file_path = audio_path + "/" + file_name
    audio_chunks=divide_audio_file(file_path, intervals_seconds, int(sample_rate/2))
    audio_chunks = audio_chunks.reshape([audio_chunks.shape[0],intervals_seconds * sample_rate,1])
    labels_array = np.ones(audio_chunks.shape[0]) * label
    predictions = model.predict(audio_chunks)
    vc = pd.DataFrame(predictions)[0].apply(lambda x: 0 if x < 0.5 else 1).value_counts()
    if (vc[0]<vc[1]):
        res = 1
    else:
        res = 0
        
    return(res)
    
    

## Get string labels from numeric ones to build a confusion matrix 

In [29]:
def getLabels(x):
    if (x>=0.5):
        label = "CY"
    else:
        label = "CN"
    return(label)
    


## Begin of the execution

In [31]:
from keras.models import load_model
model = load_model(model_path)
model.compile(optimizer='Adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

/usr/lib64/python3.6/site-packages/keras/legacy/layers.py:74: UserWarning: The `MaxoutDense` layer is deprecated and will be removed after 06/2017.
  warnings.warn('The `MaxoutDense` layer is deprecated '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 24000, 1)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 24000, 64)         8320      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 3000, 64)          0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 3000, 64, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3000, 64, 96)      5952      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 375, 64, 96)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 375, 64, 128)      430208    
__________

In [8]:
audio_path = "./data/audiodata"
labels_file = "./data/conflictlevel.csv"

intervals_seconds = 3
sample_rate = 8000

In [52]:
labels_df = pd.read_csv(labels_file, header=None)
labels_df["class"] = labels_df[1].apply(lambda x: 0 if x < 0 else 1)



In [32]:
train_df = pd.read_csv("./data/train_files.csv")
test_df = pd.read_csv("./data/test_files.csv")
validation_df = pd.read_csv("./data/validation_files.csv")

## We get into x and y the raw chunks and labels in numpy format. create_tf_records is an updated version of the original create_tf_records but whitout creating tf_files

## max_samples is the maximum number of chunks we want to predict


In [11]:
max_samples = 5000
#x_train,y_train=get_chunks_from_raw(audio_path, train_df,max_samples )
#x_test,y_test=get_chunks_from_raw(audio_path, test_df,max_samples )
#x_validation,y_validation=get_chunks_from_raw(audio_path, validation_df,max_samples )


num_samples_train = len(x_train)
num_samples_test = len(x_test)
num_samples_validation = len(x_validation)

reading file 06-05-03_600_630.wav
reading file 07-02-28_1890_1920.wav
reading file 06-05-24_1290_1320.wav
reading file 06-04-12_1110_1140.wav
reading file 07-10-31_2310_2340.wav
file 07-02-21_750_780.wav not in the dataframe
reading file 07-02-14_270_300.wav
file 06-11-22_1740_1770.wav not in the dataframe
reading file 06-11-08_480_510.wav
file 07-09-17_540_570.wav not in the dataframe
reading file 07-02-21_2070_2100.wav
reading file 06-09-13_2190_2220.wav
reading file 07-02-14_2190_2220.wav
file 08-02-06_1380_1410.wav not in the dataframe
reading file 06-04-05_1200_1230.wav
reading file 07-10-31_2280_2310.wav
file 06-04-12_2040_2070.wav not in the dataframe
reading file 07-02-28_1980_2010.wav
reading file 07-01-31_1620_1650.wav
file 07-12-19_1230_1260.wav not in the dataframe
file 06-11-22_720_750.wav not in the dataframe
reading file 06-10-18_570_600.wav
reading file 06-09-13_1140_1170.wav
reading file 06-11-08_600_630.wav
reading file 06-04-12_1710_1740.wav
reading file 07-02-14_144

file 07-01-17_990_1020.wav not in the dataframe
reading file 06-12-13_900_930.wav
reading file 06-09-13_1470_1500.wav
file 07-09-17_450_480.wav not in the dataframe
reading file 07-02-28_1680_1710.wav
file 06-05-10_1500_1530.wav not in the dataframe
reading file 07-09-17_960_990.wav
reading file 07-10-31_1650_1680.wav
file 06-05-24_660_690.wav not in the dataframe
reading file 08-01-15_1650_1680.wav
reading file 07-05-23_1860_1890.wav
reading file 06-11-22_1470_1500.wav
file 07-01-31_1380_1410.wav not in the dataframe
reading file 08-01-23_2190_2220.wav
file 08-02-06_2130_2160.wav not in the dataframe
reading file 07-10-31_2160_2190.wav
reading file 08-02-06_1860_1890.wav
reading file 06-11-08_1260_1290.wav
file 07-11-28_540_570.wav not in the dataframe
reading file 08-01-15_870_900.wav
reading file 06-05-03_1890_1920.wav
reading file 07-02-28_1260_1290.wav
file 07-01-17_1380_1410.wav not in the dataframe
reading file 06-12-20_2100_2130.wav
file 07-05-23_2160_2190.wav not in the datafr

file 07-02-14_2160_2190.wav not in the dataframe
file 07-05-02_270_300.wav not in the dataframe
file 06-10-04_1680_1710.wav not in the dataframe
file 06-11-08_1200_1230.wav not in the dataframe
file 06-09-06_330_360.wav not in the dataframe
file 08-01-23_1290_1320.wav not in the dataframe
file 07-02-28_2160_2190.wav not in the dataframe
file 06-12-13_2190_2220.wav not in the dataframe
reading file 07-02-21_1740_1770.wav
file 06-05-10_2430_2460.wav not in the dataframe
file 07-10-31_1620_1650.wav not in the dataframe
file 06-05-31_1200_1230.wav not in the dataframe
file 06-05-03_1470_1500.wav not in the dataframe
file 06-12-13_1680_1710.wav not in the dataframe
file 06-11-15_240_270.wav not in the dataframe
file 08-01-30_990_1020.wav not in the dataframe
file 07-03-28_2010_2040.wav not in the dataframe
file 06-04-05_2100_2130.wav not in the dataframe
file 07-11-14_750_780.wav not in the dataframe
file 07-09-17_870_900.wav not in the dataframe
file 07-02-21_630_660.wav not in the datafra

file 07-04-11_270_300.wav not in the dataframe
reading file 06-05-10_960_990.wav
file 07-05-16_1470_1500.wav not in the dataframe
file 06-09-06_900_930.wav not in the dataframe
file 08-01-15_1860_1890.wav not in the dataframe
file 06-09-13_840_870.wav not in the dataframe
file 07-02-28_1350_1380.wav not in the dataframe
reading file 06-11-29_270_300.wav
file 07-03-28_1050_1080.wav not in the dataframe
reading file 07-06-20_570_600.wav
file 08-01-23_1560_1590.wav not in the dataframe
file 06-10-04_1140_1170.wav not in the dataframe
reading file 06-04-05_1140_1170.wav
reading file 07-12-19_1740_1770.wav
file 06-09-13_930_960.wav not in the dataframe
file 06-09-06_780_810.wav not in the dataframe
file 06-05-17_1590_1620.wav not in the dataframe
reading file 06-04-05_1800_1830.wav
file 06-09-13_990_1020.wav not in the dataframe
file 08-01-15_1350_1380.wav not in the dataframe
file 06-05-24_990_1020.wav not in the dataframe
file 06-11-22_900_930.wav not in the dataframe
file 06-12-20_1440_1

file 06-11-29_2070_2100.wav not in the dataframe
file 06-04-05_330_360.wav not in the dataframe
file 07-01-31_570_600.wav not in the dataframe
file 08-02-06_1260_1290.wav not in the dataframe
file 06-10-04_2190_2220.wav not in the dataframe
file 06-09-13_1050_1080.wav not in the dataframe
file 06-05-10_990_1020.wav not in the dataframe
file 07-04-11_480_510.wav not in the dataframe
file 06-05-31_630_660.wav not in the dataframe
file 06-12-06_480_510.wav not in the dataframe
file 07-02-21_1950_1980.wav not in the dataframe
reading file 07-09-17_390_420.wav
file 06-12-13_840_870.wav not in the dataframe
file 06-10-04_960_990.wav not in the dataframe
file 06-12-06_390_420.wav not in the dataframe
file 07-03-28_630_660.wav not in the dataframe
file 06-09-13_1710_1740.wav not in the dataframe
file 07-02-14_1290_1320.wav not in the dataframe
file 06-09-06_660_690.wav not in the dataframe
file 06-10-11_1200_1230.wav not in the dataframe
reading file 06-09-06_2010_2040.wav
file 06-04-12_1920_1

...

file 06-09-13_840_870.wav not in the dataframe
reading file 07-02-28_1350_1380.wav
file 06-11-29_270_300.wav not in the dataframe
file 07-03-28_1050_1080.wav not in the dataframe
file 07-06-20_570_600.wav not in the dataframe
file 08-01-23_1560_1590.wav not in the dataframe
reading file 06-10-04_1140_1170.wav
file 06-04-05_1140_1170.wav not in the dataframe
file 07-12-19_1740_1770.wav not in the dataframe
reading file 06-09-13_930_960.wav
file 06-09-06_780_810.wav not in the dataframe
file 06-05-17_1590_1620.wav not in the dataframe
file 06-04-05_1800_1830.wav not in the dataframe
file 06-09-13_990_1020.wav not in the dataframe
file 08-01-15_1350_1380.wav not in the dataframe
reading file 06-05-24_990_1020.wav
file 06-11-22_900_930.wav not in the dataframe
file 06-12-20_1440_1470.wav not in the dataframe
file 06-09-13_540_570.wav not in the dataframe
file 08-01-30_1890_1920.wav not in the dataframe
file 08-01-15_1140_1170.wav not in the dataframe
file 08-01-23_1980_2010.wav not in the 

file 06-09-06_660_690.wav not in the dataframe
file 06-10-11_1200_1230.wav not in the dataframe
file 06-09-06_2010_2040.wav not in the dataframe
file 06-04-12_1920_1950.wav not in the dataframe
reading file 08-01-15_1920_1950.wav
file 07-05-30_1530_1560.wav not in the dataframe
file 07-05-02_2010_2040.wav not in the dataframe
file 06-11-08_1110_1140.wav not in the dataframe
reading file 06-11-22_1170_1200.wav
file 06-09-20_1110_1140.wav not in the dataframe
file 07-05-23_570_600.wav not in the dataframe
file 07-09-17_780_810.wav not in the dataframe
file 08-01-30_1140_1170.wav not in the dataframe
file 06-05-03_1650_1680.wav not in the dataframe
file 07-02-14_2010_2040.wav not in the dataframe
file 07-05-23_1350_1380.wav not in the dataframe
file 06-10-18_1020_1050.wav not in the dataframe
file 08-02-06_1440_1470.wav not in the dataframe
file 07-10-31_2040_2070.wav not in the dataframe
file 07-03-28_810_840.wav not in the dataframe
file 06-04-12_810_840.wav not in the dataframe
file 07

file 06-05-31_660_690.wav not in the dataframe
file 07-05-16_1650_1680.wav not in the dataframe
file 07-02-28_900_930.wav not in the dataframe
file 08-01-23_1200_1230.wav not in the dataframe
reading file 06-09-20_1320_1350.wav
file 06-12-20_1980_2010.wav not in the dataframe
file 07-09-17_840_870.wav not in the dataframe
file 07-05-30_840_870.wav not in the dataframe
file 07-02-21_780_810.wav not in the dataframe
file 08-01-23_2490_2520.wav not in the dataframe
file 06-12-06_1290_1320.wav not in the dataframe
file 07-11-14_1410_1440.wav not in the dataframe
file 06-05-03_660_690.wav not in the dataframe
reading file 06-05-24_1890_1920.wav
file 07-03-28_960_990.wav not in the dataframe
file 07-01-31_1980_2010.wav not in the dataframe
file 06-11-22_840_870.wav not in the dataframe
file 06-12-20_1530_1560.wav not in the dataframe
file 07-05-16_1050_1080.wav not in the dataframe
reading file 08-01-30_1530_1560.wav
file 06-09-13_1680_1710.wav not in the dataframe
file 07-02-14_1170_1200.wa

file 08-01-23_1680_1710.wav not in the dataframe
file 06-09-06_870_900.wav not in the dataframe
reading file 07-06-20_1170_1200.wav
file 06-12-13_2010_2040.wav not in the dataframe
file 06-05-10_2250_2280.wav not in the dataframe
file 06-09-13_1770_1800.wav not in the dataframe
reading file 07-02-28_630_660.wav
file 06-06-07_1260_1290.wav not in the dataframe
reading file 07-02-21_2010_2040.wav
file 06-10-11_1140_1170.wav not in the dataframe
file 06-06-07_2310_2340.wav not in the dataframe
file 06-09-13_1830_1860.wav not in the dataframe
reading file 07-05-16_1020_1050.wav
file 06-12-13_1290_1320.wav not in the dataframe
file 06-05-10_690_720.wav not in the dataframe
file 06-12-20_1110_1140.wav not in the dataframe
file 06-09-20_1770_1800.wav not in the dataframe
file 06-09-27_630_660.wav not in the dataframe
file 08-01-23_1920_1950.wav not in the dataframe
reading file 07-02-21_1710_1740.wav
file 06-05-31_2280_2310.wav not in the dataframe
file 07-02-28_1590_1620.wav not in the dataf

file 06-12-13_1770_1800.wav not in the dataframe
reading file 06-06-07_2670_2700.wav
file 06-05-10_750_780.wav not in the dataframe
file 07-10-31_2250_2280.wav not in the dataframe
file 07-01-31_930_960.wav not in the dataframe
file 07-04-11_1740_1770.wav not in the dataframe
file 08-01-09_990_1020.wav not in the dataframe
file 07-10-31_120_150.wav not in the dataframe
file 06-04-12_1440_1470.wav not in the dataframe
file 08-01-23_2160_2190.wav not in the dataframe
file 06-11-08_1710_1740.wav not in the dataframe
file 06-12-13_2070_2100.wav not in the dataframe
file 07-11-14_1650_1680.wav not in the dataframe
file 07-05-02_1080_1110.wav not in the dataframe
file 06-11-08_1380_1410.wav not in the dataframe
file 06-09-06_1320_1350.wav not in the dataframe
file 06-04-05_1380_1410.wav not in the dataframe
file 08-01-30_2010_2040.wav not in the dataframe
file 07-02-14_750_780.wav not in the dataframe
reading file 07-02-28_1020_1050.wav
file 07-06-20_2160_2190.wav not in the dataframe
file 0

## Prediction on train set by chunks

In [33]:
x_arr = np.concatenate(x_train).reshape([num_samples_train,24000,1])
y_arr = np.array(y_train)
print("Num samples=",num_samples_train)
print("shape X=",x_arr.shape)
print("shape Y=",y_arr.shape)


Num samples= 5009
shape X= (5009, 24000, 1)
shape Y= (5009,)


In [36]:
res=model.predict(x_arr)

In [37]:
labelTr = np.vectorize(getLabels)
pre_labels = labelTr(res)
tru_labels = labelTr(y_arr)

In [38]:
from sklearn.metrics import classification_report
metrics = classification_report(tru_labels,pre_labels)
print(metrics)

              precision    recall  f1-score   support

          CN       0.70      0.90      0.78      2461
          CY       0.86      0.62      0.72      2548

    accuracy                           0.76      5009
   macro avg       0.78      0.76      0.75      5009
weighted avg       0.78      0.76      0.75      5009



## Prediction on validation set by chunks

In [43]:
x_arr = np.concatenate(x_validation).reshape([num_samples_validation,24000,1])
y_arr = np.array(y_validation)
print("Num samples=",num_samples_validation)
print("shape X=",x_arr.shape)
print("shape Y=",y_arr.shape)



Num samples= 5010
shape X= (5010, 24000, 1)
shape Y= (5010,)


In [44]:
res=model.predict(x_arr)

In [45]:
labelTr = np.vectorize(getLabels)
pre_labels = labelTr(res)
tru_labels = labelTr(y_arr)

In [46]:
from sklearn.metrics import classification_report
metrics = classification_report(tru_labels,pre_labels)
print(metrics)

              precision    recall  f1-score   support

          CN       0.82      0.92      0.87      3246
          CY       0.81      0.62      0.70      1764

    accuracy                           0.81      5010
   macro avg       0.81      0.77      0.78      5010
weighted avg       0.81      0.81      0.81      5010



## Prediction on test set by chunks

In [48]:
x_arr = np.concatenate(x_test).reshape([num_samples_test,24000,1])
y_arr = np.array(y_test)
print("Num samples=",num_samples_test)
print("shape X=",x_arr.shape)
print("shape Y=",y_arr.shape)



Num samples= 5012
shape X= (5012, 24000, 1)
shape Y= (5012,)


In [49]:
res=model.predict(x_arr)

In [50]:
labelTr = np.vectorize(getLabels)
pre_labels = labelTr(res)
tru_labels = labelTr(y_arr)

In [51]:
from sklearn.metrics import classification_report
metrics = classification_report(tru_labels,pre_labels)
print(metrics)

              precision    recall  f1-score   support

          CN       0.73      0.90      0.80      2772
          CY       0.82      0.58      0.68      2240

    accuracy                           0.76      5012
   macro avg       0.77      0.74      0.74      5012
weighted avg       0.77      0.76      0.75      5012



## Prediction on whole audio files

In [114]:
train_df[1:10]

,0,1,class
1,06-09-06_1860_1890,0.0,1
2,07-05-02_1890_1920,1.6,1
3,08-01-30_360_390,-0.5,0
4,07-11-14_840_870,-2.1,0
5,07-02-21_270_300,0.7,1
6,06-11-08_1950_1980,1.8,1
7,06-11-29_990_1020,-3.8,0
8,07-09-17_990_1020,3.5,1
9,08-01-15_810_840,-4.8,0


In [113]:
predict_one_audio('07-11-14_840_870.wav',0.0)

0